# Check not dropping 2 stations
- I have values slightly differing from the Paper
- the only changes I undertook are dropping 2 stations due to nan values and including early stopping.
- My hypothesis is that this action might lead to a better result
- Therefore, I will check if the GNN gives the same result as in the Paper if I do not drop the 2 stations

In [6]:
%cd /home/ltchen/gnnpp
import sys
import json
import wandb
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar
from torch.optim import AdamW
from pytorch_lightning.loggers import WandbLogger
from torch_geometric.loader import DataLoader
from utils.data122 import *
from exploration.get_graphs_and_data import *


/home/ltchen/gnnpp


In [7]:
# sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
DIRECTORY = os.getcwd()
print(DIRECTORY)

leadtime = "24h"
graph_name = "g1"

# JSONPATH, SAVEPATH, RESULTPATH = get_json_save_result_paths(leadtime=leadtime, graph_name=graph_name)

/home/ltchen/gnnpp


In [8]:
dataframes = load_dataframes_1(leadtime=leadtime)
dataframes = summary_statistics(dataframes)

[INFO] Dataframes exist. Will load pandas dataframes.
[INFO] Calculating summary statistics for train
[INFO] Calculating summary statistics for valid
[INFO] Calculating summary statistics for test_rf
[INFO] Calculating summary statistics for test_f


In [9]:
dataframes['train'][0].nunique()

time                  2612
station_id             122
model_orography        116
station_altitude       118
station_latitude       122
                     ...  
v_mean              306905
v_std               308216
t_mean              248620
t_std               308216
number                   1
Length: 65, dtype: int64

In [5]:
graph_name = "g1"
drn = False
if drn == True:
    SAVEPATH = os.path.join(DIRECTORY, f"leas_trained_models/drn_{leadtime}/models")
    JSONPATH = os.path.join(DIRECTORY, f"trained_models/drn_{leadtime}/params.json")
else:
    SAVEPATH = os.path.join(DIRECTORY, f"leas_trained_models/sum_stats_{leadtime}/122{graph_name}_{leadtime}/models")
    JSONPATH = os.path.join(DIRECTORY, f"leas_trained_models/sum_stats_{leadtime}/{graph_name}_{leadtime}/params.json")
    RESULTPATH = os.path.join(DIRECTORY, f"leas_trained_models/sum_stats_{leadtime}/122{graph_name}_{leadtime}")


with open(JSONPATH, "r") as f:
    print(f"[INFO] Loading {JSONPATH}")
    args_dict = json.load(f)
config = args_dict

print(SAVEPATH)
print(JSONPATH)
print(RESULTPATH)

[INFO] Loading /home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/g1_24h/params.json
/home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/122g1_24h/models
/home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/g1_24h/params.json
/home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/122g1_24h


In [6]:
graphs1_train_rf, tests1 = normalize_features_and_create_graphs1(df_train=dataframes['train'], df_valid_test=[dataframes['valid'], dataframes['test_rf'], dataframes['test_f']], station_df=dataframes['stations'], attributes=["geo"], edges=[("geo", 50)], sum_stats = True)
graphs1_valid_rf, graphs1_test_rf, graphs1_test_f = tests1

g1_train_loader = DataLoader(graphs1_train_rf, batch_size=config['batch_size'], shuffle=True)
g1_valid_loader = DataLoader(graphs1_valid_rf, batch_size=config['batch_size'], shuffle=True)
g1_test_f_loader = DataLoader(graphs1_test_f, batch_size=config['batch_size'], shuffle=False)
g1_test_rf_loader = DataLoader(graphs1_test_rf, batch_size=config['batch_size'], shuffle=False)

train_loader = g1_train_loader
valid_loader = g1_valid_loader
test_f_loader = g1_test_f_loader
test_rf_loader = g1_test_rf_loader
test_loader = [test_f_loader, test_rf_loader]

emb_dim = 20
in_channels = graphs1_train_rf[0].x.shape[1] + emb_dim - 1
edge_dim = graphs1_train_rf[0].num_edge_features
# max_epochs = 100

[INFO] Normalizing features...
fit_transform
transform 1
transform 2
transform 3


100%|██████████| 730/730 [00:02<00:00, 260.45it/s]


In [7]:
max_epochs = config['max_epochs']
num_nodes = graphs1_train_rf[0].num_nodes
print(max_epochs)
print(num_nodes)

23
122


In [10]:
print(load_distances_1(dataframes['stations']).shape)

[INFO] Computing distances...
(122, 122)


In [8]:
PROJECTNAME = "g122_run1"

for i in range(5, 11):
    TRAINNAME = f"{graph_name}_{leadtime}_train_run{i}_max_epoch23"

    with wandb.init(
            project=PROJECTNAME, id=TRAINNAME, config=args_dict, tags=["check_122"], resume="never"
    ):
        config = wandb.config

        multigraph = Multigraph(
            num_nodes=num_nodes, #
            embedding_dim=emb_dim,
            edge_dim=edge_dim,
            in_channels=in_channels,
            hidden_channels_gnn=config['gnn_hidden'],
            out_channels_gnn=config['gnn_hidden'],
            num_layers_gnn=config['gnn_layers'],
            heads=config['heads'],
            hidden_channels_deepset=config['gnn_hidden'],
            optimizer_class=AdamW,
            optimizer_params=dict(lr=config['lr']),
        )
        torch.compile(multigraph)
        batch = next(iter(train_loader))
        multigraph.forward(batch)

        wandb_logger = WandbLogger(project=PROJECTNAME)
        # early_stop = EarlyStopping(monitor="val_loss", patience=10)
        progress_bar = TQDMProgressBar(refresh_rate=0)

        checkpoint_callback = ModelCheckpoint(
            dirpath=SAVEPATH, filename=TRAINNAME, monitor="val_loss", mode="min", save_top_k=1
        )

        trainer = L.Trainer(
                max_epochs=max_epochs,
                log_every_n_steps=1,
                accelerator="gpu",
                devices = 1,
                enable_progress_bar=True,
                logger=wandb_logger,
                # callbacks=[early_stop, progress_bar, checkpoint_callback],
                callbacks=[checkpoint_callback, progress_bar],
        )

        trainer.fit(model=multigraph, train_dataloaders=train_loader, val_dataloaders=valid_loader)

wandb: Currently logged in as: leachen01 (leachen01-karlsruhe-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/122g1_24h/models exists and is not empty.
LOC

epoch,▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇█████
train_loss_epoch,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇████
val_loss,█▄▃▂▃▂▂▃▂▂▁▂▁▂▁▁▁▁▁▁▁▃▂
epoch,22
train_loss_epoch,0.6017
train_loss_step,0.60181
trainer/global_step,7520
val_loss,0.68253


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/122g1_24h/models exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type              | Params | Mode 
----------------------------------------------------------
0 | encoder     | EmbedStations     | 2.4 K  | train
1 | conv        | ResGnn            | 878 K  | train
2 | aggr        | DeepSetAggregator | 197 K  | train
3 | postprocess | MakePositive      | 0     

epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇█████
train_loss_epoch,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▅▅▅▅▄▃▆▆▃▃▃▄▂▂▂▂▃▂▃▄▁▂▂▄▄▃▁▃▂▃▃▄▃▁▃▂▂▂▂
trainer/global_step,▁▁▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇█
val_loss,█▅▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch,22
train_loss_epoch,0.60709
train_loss_step,0.57233
trainer/global_step,7520
val_loss,0.66634


...

epoch,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train_loss_epoch,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█
val_loss,█▄▃▃▃▂▃▂▂▂▁▂▂▁▁▁▁▂▁▁▂▁▂
epoch,22
train_loss_epoch,0.60666
train_loss_step,0.60735
trainer/global_step,7520
val_loss,0.70218


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/122g1_24h/models exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type              | Params | Mode 
----------------------------------------------------------
0 | encoder     | EmbedStations     | 2.4 K  | train
1 | conv        | ResGnn            | 878 K  | train
2 | aggr        | DeepSetAggregator | 197 K  | train
3 | postprocess | MakePositive      | 0     

epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇██
train_loss_epoch,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▇▆▄▅▆▄▃▅▃▃▃▃▂▃▃▃▂▃▁▃▃▄▃▃▁▁▂▁▃▁▂▂▃▂▂▁▁▂▄
trainer/global_step,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█████
val_loss,█▄▃▃▄▂▃▂▃▂▂▂▁▂▁▁▁▁▁▁▁▂▁
epoch,22
train_loss_epoch,0.60573
train_loss_step,0.63176
trainer/global_step,7520
val_loss,0.66172


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/122g1_24h/models exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type              | Params | Mode 
----------------------------------------------------------
0 | encoder     | EmbedStations     | 2.4 K  | train
1 | conv        | ResGnn            | 878 K  | train
2 | aggr        | DeepSetAggregator | 197 K  | train
3 | postprocess | MakePositive      | 0     

epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▇▇▇▇█
train_loss_epoch,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▆▇█▆▅▂▄▄▅▃▃▃▃▂▃▃▃▅▃▄▃▃▂▃▂▅▁▂▄▂▃▃▃▅▂▁▃▂▃
trainer/global_step,▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
val_loss,█▅▃▃▃▃▂▂▃▁▂▁▁▁▁▁▁▁▁▁▂▂▁
epoch,22
train_loss_epoch,0.6085
train_loss_step,0.67044
trainer/global_step,7520
val_loss,0.66827


In [9]:
data_list = ["f", "rf"]
print(SAVEPATH)
for data, tl in zip(data_list, test_loader):
    preds_list = []
    for path in os.listdir(SAVEPATH):
        if path.endswith(".ckpt"):
            print(f"[INFO] Loading model from {path}")
            # Load Model from checkpoint

            multigraph = Multigraph.load_from_checkpoint(
                os.path.join(SAVEPATH, path),
                num_nodes=num_nodes,
                embedding_dim=emb_dim,
                edge_dim=edge_dim,
                in_channels=in_channels,
                hidden_channels_gnn=config['gnn_hidden'],
                out_channels_gnn=config['gnn_hidden'],
                num_layers_gnn=config['gnn_layers'],
                heads=config['heads'],
                hidden_channels_deepset=config['gnn_hidden'],
                optimizer_class=AdamW,
                optimizer_params=dict(lr=config['lr']),
            )
            multigraph.eval()
            batch = next(iter(train_loader))
            batch = batch.to("cuda")
            multigraph.to("cuda")
            multigraph.forward(batch)

            trainer = L.Trainer(log_every_n_steps=1, accelerator="gpu", devices=[1], enable_progress_bar=True)

            ####################################################################################################
            preds = trainer.predict(model=multigraph, dataloaders=[tl])
            preds = torch.cat(preds, dim=0)
            preds_list.append(preds)
            print()
            print(preds.shape)

    targets = dataframes[f"test_{data}"][1]
    targets = torch.tensor(targets.t2m.values) - 273.15

    stacked = torch.stack(preds_list)
    final_preds = torch.mean(stacked, dim=0)

    res = multigraph.loss_fn.crps(final_preds, targets)
    print("#############################################")
    print("#############################################")
    print(f"final crps for {data}: {res.item()}")
    print("#############################################")
    print("#############################################")

    ####################################################################################################
    os.makedirs(RESULTPATH, exist_ok=True)
    print(RESULTPATH)

    df = pd.DataFrame(np.concatenate([targets.view(-1, 1), final_preds], axis=1), columns=["t2m", "mu", "sigma"])
    df.to_csv(os.path.join(RESULTPATH, f"{data}_{graph_name}_{leadtime}_results.csv"), index=False)

    # Create Log File ###############################################################
    log_file = os.path.join(RESULTPATH, f"{data}.txt")
    with open(log_file, "w") as f:
        f.write(f"Data: {data}\n")
        f.write(f"Leadtime: {leadtime}\n")
        f.write(f"Final crps: {res.item()}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float

/home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/122g1_24h/models
[INFO] Loading model from g1_24h_train_run5_max_epoch23.ckpt
Predicting DataLoader 0: 100%|██████████| 92/92 [00:00<00:00, 144.24it/s]

torch.Size([89060, 2])
[INFO] Loading model from g1_24h_train_run9_max_epoch23.ckpt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting DataLoader 0: 100%|██████████| 92/92 [00:00<00:00, 179.42it/s]

torch.Size([89060, 2])
[INFO] Loading model from g1_24h_train_run3_max_epoch23.ckpt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

Predicting DataLoader 0: 100%|██████████| 92/92 [00:00<00:00, 151.26it/s]

torch.Size([89304, 2])
[INFO] Loading model from g1_24h_train_run10_max_epoch23.ckpt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting DataLoader 0: 100%|██████████| 92/92 [00:00<00:00, 161.66it/s]

torch.Size([89304, 2])
[INFO] Loading model from g1_24h_train_run2_max_epoch23.ckpt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting DataLoader 0: 100%|██████████| 92/92 [00:00<00:00, 183.69it/s]

torch.Size([89304, 2])
#############################################
#############################################
final crps for rf: 0.6250079273463535
#############################################
#############################################
/home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/122g1_24h


In [14]:
data_list = ["f", "rf"]
print(SAVEPATH)
for data, tl in zip(data_list, test_loader):
    preds_list = []
    for path in os.listdir(SAVEPATH):
        if path.endswith(".ckpt"):
            print(f"[INFO] Loading model from {path}")
            # Load Model from checkpoint

            multigraph = Multigraph.load_from_checkpoint(
                os.path.join(SAVEPATH, path),
                num_nodes=num_nodes,
                embedding_dim=emb_dim,
                edge_dim=edge_dim,
                in_channels=in_channels,
                hidden_channels_gnn=config['gnn_hidden'],
                out_channels_gnn=config['gnn_hidden'],
                num_layers_gnn=config['gnn_layers'],
                heads=config['heads'],
                hidden_channels_deepset=config['gnn_hidden'],
                optimizer_class=AdamW,
                optimizer_params=dict(lr=config['lr']),
            )
            multigraph.eval()
            batch = next(iter(train_loader))
            batch = batch.to("cuda")
            multigraph.to("cuda")
            multigraph.forward(batch)

            trainer = L.Trainer(log_every_n_steps=1, accelerator="gpu", devices=[1], enable_progress_bar=False)

            ####################################################################################################
            preds = trainer.predict(model=multigraph, dataloaders=[tl])
            preds = torch.cat(preds, dim=0)
            # preds_list.append(preds)
            print()
            print(preds.shape)

            targets = dataframes[f"test_{data}"][1]
            targets = torch.tensor(targets.t2m.values) - 273.15

            #stacked = torch.stack(preds_list)
            #final_preds = torch.mean(stacked, dim=0)

            res = multigraph.loss_fn.crps(preds, targets).item()
            # print(res)
            preds_list.append(res)

    print(len(preds_list))
    mean = sum(preds_list) / len(preds_list)
    print("#############################################")
    print("#############################################")
    print(f"mean crps for {data}: {mean}")
    print("#############################################")
    print("#############################################")

    ####################################################################################################
'''
    os.makedirs(RESULTPATH, exist_ok=True)
    print(RESULTPATH)

    df = pd.DataFrame(np.concatenate([targets.view(-1, 1), final_preds], axis=1), columns=["t2m", "mu", "sigma"])
    df.to_csv(os.path.join(RESULTPATH, f"{data}_{graph_name}_{leadtime}_results.csv"), index=False)

    # Create Log File ###############################################################
    log_file = os.path.join(RESULTPATH, f"{data}.txt")
    with open(log_file, "w") as f:
        f.write(f"Data: {data}\n")
        f.write(f"Leadtime: {leadtime}\n")
        f.write(f"Final crps: {res.item()}")
'''

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


/home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/122g1_24h/models
[INFO] Loading model from g1_24h_train_run5_max_epoch23.ckpt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]



torch.Size([89060, 2])
[INFO] Loading model from g1_24h_train_run9_max_epoch23.ckpt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]



torch.Size([89304, 2])
[INFO] Loading model from g1_24h_train_run10_max_epoch23.ckpt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]



torch.Size([89304, 2])
[INFO] Loading model from g1_24h_train_run2_max_epoch23.ckpt

torch.Size([89304, 2])
10
#############################################
#############################################
mean crps for rf: 0.6443902809471849
#############################################
#############################################


'\n    os.makedirs(RESULTPATH, exist_ok=True)\n    print(RESULTPATH)\n\n    df = pd.DataFrame(np.concatenate([targets.view(-1, 1), final_preds], axis=1), columns=["t2m", "mu", "sigma"])\n    df.to_csv(os.path.join(RESULTPATH, f"{data}_{graph_name}_{leadtime}_results.csv"), index=False)\n\n    # Create Log File ###############################################################\n    log_file = os.path.join(RESULTPATH, f"{data}.txt")\n    with open(log_file, "w") as f:\n        f.write(f"Data: {data}\n")\n        f.write(f"Leadtime: {leadtime}\n")\n        f.write(f"Final crps: {res.item()}")\n'

In [11]:
f_df = pd.read_csv(os.path.join(RESULTPATH, f"f_{graph_name}_{leadtime}_results.csv"))
f_df

,t2m,mu,sigma
0,5.5,5.584488,0.870826
1,2.1,1.735488,0.907235
2,6.6,5.977023,0.867696
3,6.0,5.880330,0.909379
4,1.9,1.077887,0.933794
...,...,...,...
89055,4.8,5.256744,1.192812
89056,3.7,4.261754,1.431870
89057,3.2,3.373278,1.461015
89058,-2.5,-1.312616,1.152949


In [26]:
data_list = ["f", "rf"]
print(SAVEPATH)
for data, tl in zip(data_list, test_loader):
    preds_list = []
    # for path in os.listdir(SAVEPATH):
        # if path.endswith(".ckpt"):
    path = os.path.join(SAVEPATH, f"{graph_name}_{leadtime}_train_run1-v1.ckpt")
    print(f"[INFO] Loading model from {path}")
    # Load Model from checkpoint

    multigraph = Multigraph.load_from_checkpoint(
        os.path.join(SAVEPATH, path),
        num_nodes=num_nodes,
        embedding_dim=emb_dim,
        edge_dim=edge_dim,
        in_channels=in_channels,
        hidden_channels_gnn=config['gnn_hidden'],
        out_channels_gnn=config['gnn_hidden'],
        num_layers_gnn=config['gnn_layers'],
        heads=config['heads'],
        hidden_channels_deepset=config['gnn_hidden'],
        optimizer_class=AdamW,
        optimizer_params=dict(lr=config['lr']),
    )
    multigraph.eval()
    batch = next(iter(train_loader))
    batch = batch.to("cuda")
    multigraph.to("cuda")
    multigraph.forward(batch)

    trainer = L.Trainer(log_every_n_steps=1, accelerator="gpu", devices=[1], enable_progress_bar=True)

    ####################################################################################################
    preds = trainer.predict(model=multigraph, dataloaders=[tl])
    preds = torch.cat(preds, dim=0)
    preds_list.append(preds)
    print()
    print(preds.shape)

    targets = dataframes[f"test_{data}"][1]
    targets = torch.tensor(targets.t2m.values) - 273.15

    stacked = torch.stack(preds_list)
    final_preds = torch.mean(stacked, dim=0)

    res = multigraph.loss_fn.crps(final_preds, targets)
    print("#############################################")
    print("#############################################")
    print(f"final crps for {data}: {res.item()}")
    print("#############################################")
    print("#############################################")

    ####################################################################################################
    os.makedirs(RESULTPATH, exist_ok=True)
    print(RESULTPATH)

    df = pd.DataFrame(np.concatenate([targets.view(-1, 1), final_preds], axis=1), columns=["t2m", "mu", "sigma"])
    df.to_csv(os.path.join(RESULTPATH, f"{data}_{graph_name}_{leadtime}_results.csv"), index=False)

    # Create Log File ###############################################################
    log_file = os.path.join(RESULTPATH, f"{data}.txt")
    with open(log_file, "w") as f:
        f.write(f"Data: {data}\n")
        f.write(f"Leadtime: {leadtime}\n")
        f.write(f"Final crps: {res.item()}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


/home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/122g1_24h/models
[INFO] Loading model from /home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/122g1_24h/models/g1_24h_train_run1-v1.ckpt
Predicting DataLoader 0: 100%|██████████| 92/92 [00:00<00:00, 158.85it/s]

torch.Size([89060, 2])
#############################################
#############################################
final crps for f: 0.6443530764200678
#############################################
#############################################
/home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/122g1_24h


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


[INFO] Loading model from /home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/122g1_24h/models/g1_24h_train_run1-v1.ckpt
Predicting DataLoader 0: 100%|██████████| 92/92 [00:00<00:00, 159.33it/s]

torch.Size([89304, 2])
#############################################
#############################################
final crps for rf: 0.6451957248009318
#############################################
#############################################
/home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/122g1_24h
